<a href="https://colab.research.google.com/github/Rishika70/Hack_To_Hire/blob/main/Hack_To_Hire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

In [10]:
!pip install bleu

  Preparing metadata (setup.py) ... done
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5780 sha256=ce717a0eb9c44bc76db46b203c8e32eca926ebfead538e0ff9f22239876579cf
  Stored in directory: /root/.cache/pip/wheels/c6/d8/d1/009abe01b8b2c6a14c62d197b510b3cc1076014c22d712c5ce
Successfully built bleu


In [19]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel
from transformers import T5Tokenizer, T5Model
from transformers import GPT2Tokenizer, GPT2Model
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split


In [20]:
# Load and split the dataset
dataset = load_dataset("toughdata/quora-question-answer-dataset")
dataset = dataset["train"].train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]



In [16]:
# Tokenize the data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [21]:

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the 'question' and 'answer' columns
dataset = dataset.map(lambda x: {'question': preprocess_text(x['question']),
                                 'answer': preprocess_text(x['answer'])})

# Convert dataset to pandas DataFrame for splitting
df = dataset['train'].to_pandas()

# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

print("Training Data:")
print(train_data)
print("\nValidation Data:")
print(val_data)


Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

Map:   0%|          | 0/11281 [00:00<?, ? examples/s]

Training Data:
                                                question  \
17394                                 short story wife ?   
29099          Could Data pas Kobayashi Maru Star Trek ?   
42067                                   mean Canadians ?   
24706          write speech `` pet people n't child '' ?   
35644                                              way ?   
...                                                  ...   
11284  Melinda Gates say choosing divorce Bill 'lowes...   
44732  Would Hillary Clinton made better President Do...   
38158          video game 90 made ? kind software used ?   
860    bank teller allowed ask withdrawing lot cash a...   
15795                             underrated city U.S. ?   

                                                  answer  
17394  wife suspected husband cheating one fine day h...  
29099  script written character passed test—YES scrip...  
42067              Definitely yes . country politician .  
24706       title speech “ l

In [23]:
# To display the entirety of each row in the Dataset
pd.set_option('display.max_colwidth', None)

In [22]:
df.head()


,question,answer
0,best thing die ?,join ART LIVING . various course like happines...
1,misunderstood picture become famous ( infamous...,Napalm Girl Nick Ut picture immediately came m...
2,best way get discount hotel room ?,Otel fırsatlarını indirimlerini arayın . Birço...
3,best way invest money ?,various way invest money sure minimize risk ri...
4,calculate,One quick way calculate 1+2+3+4+5+6+7+8+9+10 u...


Dataset Rows & Columns count

In [24]:
# Dataset Rows & Columns count
df.shape

(45121, 2)

Dataset Information

In [25]:
# Dataset Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45121 entries, 0 to 45120
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  45121 non-null  object
 1   answer    45121 non-null  object
dtypes: object(2)
memory usage: 705.1+ KB


Duplicate Values

In [26]:
# Dataset Duplicate Value Count
df.duplicated().sum()

897

Missing Values/Null Values

In [27]:
# Missing Values/Null Values Count
df.isna().sum()

question    0
answer      0
dtype: int64